20 most populous cities in Canada are:

Toronto
Montreal
Vancouver
Calgary
Edmonton
Ottawa-Gatineau
Winnipeg
Quebec City
Hamilton
Kitchener-Waterloo-Cambridge
London
Victoria
Halifax
Oshawa
Windsor
Saskatoon
St Catherine's-Niagara
Regina
St John's
Kelowna

In [3]:
from IPython.display import HTML
import plotly.offline as py
import plotly.graph_objs as go
import numpy as np
import random
import csv

py.init_notebook_mode(connected=True)

# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# The raw code for this cell is by default hidden for easier reading.
# To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')


# 3/8/18 For some reason I forgot to put in Victoria. I'm not going to fix it right now.

In [83]:
def read_lines(filename, skip = False):
    """ Function to read in CSV file as numerical data. """
    with open(filename) as data:
        reader = csv.reader(data)
        if skip:  # Skip the header if it's there.
            next(reader, None)
            for row in reader:
                yield [ float(i) for i in row ]
        else:
            for row in reader:
                yield [ float(i) for i in row ]

# Names of the cities we'll be visiting.
city_names = ['tor','mtl','van','cgy','edm','ott','wpg','qbc','ham','kwc','ldn','hfx','osh','win','sks','stc','reg','stj','kel']

city_data = [row for row in read_lines('data/city_data.csv', True)]

# This constructs a city with the numerical data in city_data.
def Cities(city_names):
    def City(city_names, row_index):
        N = len(city_names)

        city = {}
        for index in range(N):
            city[city_names[index]] = city_data[row_index][index]

        return city

    # This builds the dictionary of cities that we want.
    cities = {}

    for i in range(len(city_names)):
        c = city_names[i]
        cities[c] = City(city_names,i)

    return cities

def Coordinates(city_names):
    """ Function to create a dictionary of dictionaries. Each dictionary 
        contains coordinates for a given city. """

    T = [i for i in read_lines('data/coordinates.csv')]

    """ This line returns a dictionary with each city name as the key. Each 
        key has a dictionary as its value. This second dictionary contains
        the coordinates for the respective city. """
    F = {city_names[i] : {'lat': T[i][0], 'lon': T[i][1]} for i in range(len(city_names))}

    return F

class Trip:
    """ Class for performing a trip. """
    
    def __init__(self, city_names):
        self.cities = Cities(city_names)
        self.total_distance = 0
        self.coordinates = Coordinates(city_names)
        self.latitudes = None
        self.longitudes = None
        
        # This saves the order in which the cities were visited.
        self.order = []
        
    def setup(self):
        """ Function to select a random starting point. """
        self.starting_city = random.choice(list(self.cities.keys()))
#         print("We're starting in " + self.starting_city)
        
    def shortest(self):
        return min(self.cities[self.starting_city], key = self.cities[self.starting_city].get)

    def update(self):
        """ Function to move between cities. """
        # We don't want the starting city to be a destination for the other cities anymore.
        for key in self.cities.keys():
            self.cities[key].pop(self.starting_city)
        
        # Finds the closest city to the current city.
        if self.cities[self.starting_city] is not None:
            closest = self.shortest()
        
        # Add to the total distance.
        self.total_distance += self.cities[self.starting_city][closest]
        
#         print("We're going from " + self.starting_city + " to " + closest)
#         print(self.total_distance)

        self.order.append(self.coordinates[self.starting_city])
        self.starting_city = closest
    
    def trip(self):
        """ Function to carry out a complete trip using recursion. """
        if len(self.cities[self.starting_city].keys()) > 1:
            self.update()
            self.trip()
        else:
            self.order.append(self.coordinates[self.starting_city])
            self.latitudes = [value['lat'] for value in self.order]
            self.longitudes = [value['lon'] for value in self.order]
            print('The total distance for this trip was {}.'.format(self.total_distance))
            
    def plot_trip(self):
        """ Function to plot the trip interactively with plotly. """
        journey = [ dict(
            type = 'scattergeo',
            name = "Path taken",
            lat = self.latitudes,
            lon = self.longitudes,
#             hoverinfo = 'text',
#             text = ['Rodez, France', 'Barcelona, Spain'],
            mode = 'lines',
            line = dict(
                width = 2,
                color = 'rgb(0,0,0)',
            ),
        ) ]

        layout1 = dict(
                title = 'Traveling Salesperson Trip',
                showlegend = False,         
                geo = dict(
                    resolution = 50,
                    showland = True,
                    showlakes = True,
                    showocean = True,
                    landcolor = 'rgb(0, 179, 0)',
                    lakecolor = 'rgb(214, 224, 245)',
                    oceancolor = 'rgb(214, 224, 245)',
                    projection = dict( type="mercator" ),
                    coastlinewidth = 1,
                    lataxis = dict(
                        range = [ 40, 65 ],
                        showgrid = True,
                        tickmode = "linear",
                        dtick = 10
                    ),
                    lonaxis = dict(
                        range = [-135, -45],
                        showgrid = True,
                        tickmode = "linear",
                        dtick = 10
                    ),
                )
            )

        fig = dict(data = journey, layout=layout1)

        py.iplot(fig, validate=False)

In [84]:
def random_trips(N):
    """ Take N trips and find the best distance. """
    # Initialize N trips.
    trips = [Trip(city_names) for i in range(N)]
    
    for trip in trips:
        trip.setup()
        trip.trip()
    
    distances = [trip.total_distance for trip in trips]
    
    shortest_trip = min(distances)
    result = trips[trip.total_distance == shortest_trip]
    
    print('The shortest trip was {} km.'.format(shortest_trip))
    print(result)
    result.plot_trip()
    
    

# c = Trip(city_names)
# c.setup()
# c.trip()
random_trips(20)

The total distance for this trip was 8035.0.
The total distance for this trip was 13833.0.
The total distance for this trip was 8035.0.
The total distance for this trip was 13659.0.
The total distance for this trip was 13177.0.
The total distance for this trip was 8390.0.
The total distance for this trip was 11151.0.
The total distance for this trip was 13833.0.
The total distance for this trip was 10555.0.
The total distance for this trip was 11151.0.
The total distance for this trip was 11467.0.
The total distance for this trip was 10171.0.
The total distance for this trip was 11467.0.
The total distance for this trip was 10953.0.
The total distance for this trip was 10953.0.
The total distance for this trip was 13177.0.
The total distance for this trip was 11103.0.
The total distance for this trip was 13659.0.
The total distance for this trip was 13761.0.
The total distance for this trip was 11103.0.
The shortest trip was 8035.0 km.
